In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import statistics
import os
import random
from mymodels import ResnetOCTModel, ImageDataset, filter_by_lens, get_paths_and_labels, split_subjects, construct_dataset

/Users/elvisyan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
lens = 12.6 
data = pd.read_excel('OCT_Collated.xlsx')
data = data.loc[:, ['Subject (Eye)','ICL Size', 'Post-Op Vault']]
data = data[-data['Post-Op Vault'].isna()]
if lens: 
        data = filter_by_lens(data, lens)
subject_test, subject_train, subject_val = split_subjects(data)
dataloader_test, dataloader_train, dataloader_val = construct_dataset(data, subject_test, subject_train, subject_val)

selected subjects: 36 163 41
datasets constructed. test length: 225, train length 414, val length 126


In [8]:
for epoch in [16, 32, 64, 128, 256]:
    for lr in [0.001, 0.005, 0.01]:
        for dropout in [0.2, 0.3, 0.4, 0.5]:
            for freeze in [False, True]:
                print('current setting:', epoch, lr, dropout, freeze)
                curr_model = ResnetOCTModel([], dropout, freeze)
                curr_model.train_model(epoch, dataloader_train, dataloader_val, learning_rate=lr, early_stop=False, graph=True)
                curr_model.evaluate(dataloader_test)
                torch.save(curr_model, f"new_trained_models/model{epoch}_{lr}_{dropout}_{freeze}")

current setting: 16 0.001 0.2 False


/Users/elvisyan/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/elvisyan/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


training start
----------------------
Epoch 1/16, Traning Loss: 187.19586929908166, Validation loss: 201.93671321868896
Epoch 2/16, Traning Loss: 149.97593461550198, Validation loss: 139.076078414917
Epoch 3/16, Traning Loss: 152.58770121060886, Validation loss: 128.1584758758545
Epoch 4/16, Traning Loss: 153.41615383441632, Validation loss: 152.72262740135193
Epoch 5/16, Traning Loss: 151.95941462883582, Validation loss: 125.66236066818237
Epoch 6/16, Traning Loss: 153.99134870675894, Validation loss: 131.17600440979004
Epoch 7/16, Traning Loss: 152.40885411776028, Validation loss: 131.50299739837646
